In [1]:
import numpy as np
import sympy as sp
from scipy.integrate import quad

from tqdm.notebook import tqdm
from time import sleep

import seaborn
import matplotlib.pyplot as plt

In [12]:
# Define the symbols
alpha, beta = sp.symbols('alpha beta', complex=True)
a, b, c = sp.symbols('a b c', real=True, positive=True)

# Construct the Sigma matrix
Sigma = sp.Matrix([
    [a, 0, c, 0],
    [0, a, 0, -c],
    [c, 0, b, 0],
    [0, -c, 0, b]
])

# Calculate the determinant of Sigma
det_Sigma = Sigma.det()

# Define the magnitude and argument functions
abs_alpha = sp.Abs(alpha)
abs_beta = sp.Abs(beta)
arg_alpha = sp.arg(alpha)
arg_beta = sp.arg(beta)

# Construct the expression for Q(alpha, beta)
Q = (sp.sqrt(det_Sigma) / sp.pi**2) * sp.exp(-a * abs_alpha**2 - b * abs_beta**2 - 2 * c * abs_alpha * abs_beta * sp.cos(arg_alpha - arg_beta))

Q

exp(-a*Abs(alpha)**2 - b*Abs(beta)**2 - 2*c*cos(arg(alpha) - arg(beta))*Abs(alpha)*Abs(beta))*Abs(a*b - c**2)/pi**2

In [16]:
# Define the integration variables. That is, the real and imaginary parts of alpha and beta
alpha_re, alpha_im, beta_re, beta_im = sp.symbols('alpha_re alpha_im beta_re beta_im', real=True)

# Substitute in the actual values to integrate over
Q_re_and_im = Q.subs([(alpha, alpha_re + sp.I * alpha_im), (beta, beta_re + sp.I * beta_im)])

# Symbolically integrate over beta_re, beta_im and alpha im
Q_alpha_re = sp.integrate(Q_re_and_im, (beta_re, -sp.oo, sp.oo), (beta_im, -sp.oo, sp.oo), (alpha_im, -sp.oo, sp.oo))

Q_alpha_re

exp(-a*alpha_re**2)*Abs(a*b - c**2)*Integral(exp(-a*alpha_im**2)*Integral(exp(-b*beta_im**2)*Integral(exp(-b*beta_re**2)*exp(-2*c*sqrt(alpha_im**2 + alpha_re**2)*sqrt(beta_im**2 + beta_re**2)*cos(arg(I*alpha_im + alpha_re) - arg(I*beta_im + beta_re))), (beta_re, -oo, oo)), (beta_im, -oo, oo)), (alpha_im, -oo, oo))/pi**2

In [17]:
# Lambdify the expression for numerical evaluation, for a given a, b and c. For a = 1, b = 1 and c = 0, this should reduce to a Gaussian integral. For any parameter values, the result should be 1.
Q_alpha_re_lambdified = sp.lambdify(alpha_re, Q_alpha_re.subs([(a, 1), (b, 1), (c, 0)]).simplify(), 'numpy')